# Import

In [2]:
import json as js
import pandas as pd
import numpy as np
import tensorflow as tf
import sys
import traceback
import re

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from urllib.request import urlopen
from imageio import imread

import cv2
import tensorflow.keras as keras
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import decode_predictions\

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from ipywidgets import IntProgress
from IPython.display import display

from preprocessing import preprocess_customer, preprocess_reactions

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

import catboost
from sklearn.metrics import accuracy_score, confusion_matrix

In [3]:
path = './data/'

test_info = pd.read_csv(path + 'customer_test.csv') # customer
train_info = pd.read_csv(path + 'customer_train.csv') # customer
react_train = pd.read_csv(path + 'stories_reaction_train.csv')
react_test = pd.read_csv(path + 'stories_reaction_test.csv')
sample = pd.read_csv(path + 'sample_submit.csv')

transactions = pd.read_csv(path + 'restructed_transactions.csv')
stories_names = pd.read_csv(path + 'restructed_stories_description.csv')

In [4]:
cust_one = preprocess_customer(
    train_info, react_train, encodings=['frequency', 'one-hot'])

cust_two = preprocess_customer(
    test_info, react_test, encodings=['frequency', 'one-hot'])

cust = pd.concat([cust_one, cust_two]).set_index('customer_id').join(transactions.set_index('Unnamed: 0'), on = 'customer_id').fillna(0)

data_train, data_test = preprocess_reactions(
    react_train, react_test, encodings = ['mean', 'frequency'])

data_train = cust.join(data_train.set_index('customer_id'), on = 'customer_id')
data_test = cust.join(data_test.set_index('customer_id'), on = 'customer_id')

# add labels of images
data_train = data_train.set_index('story_id').join(
    stories_names.drop('image_data', axis=1).set_index('story_id'), 
    on = 'story_id')
data_train['name_data'].fillna("", inplace=True)

data_test = data_test.set_index('story_id').join(
    stories_names.drop('image_data', axis=1).set_index('story_id'),
    on = 'story_id')
data_test['name_data'].fillna("", inplace=True)

del cust, cust_one, cust_two, test_info, train_info, react_train, react_test, stories_names

like, view, skip, dislike =  [1 3 2 0]


In [5]:
data_train = data_train.set_index(pd.Index([i for i in range(data_train.shape[0])]))
data_test = data_test.set_index(pd.Index([i for i in range(data_test.shape[0])]))

In [6]:
a_=0
b_=0
c_=0
d_=0
e_=0
def inition(start, end, data, ev=True):
    global a_, b_, c_, d_, e_
    charisma = data[[
        'gender_cd',
        'children_cnt_freq',
        'customer_id_freq',
        'product_0_freq',
        'product_1_freq',
        'product_2_freq',
        'product_3_freq',
        'product_4_freq',
        'product_5_freq',
        'product_6_freq',
    ]][start:end]
    a_ = len(charisma.columns)

    charisma_ones = data[[
        'ft_month1',
        'ft_month2',
        'ft_month3',
        'ft_month4',
        'ft_month5',
        'ft_month6',
        'ft_month7',
        'ft_month8',
        'ft_month9',
        'ft_month10',
        'ft_month11',
        'ft_month12',
        'ft_year2012',
        'ft_year2013',
        'ft_year2014',
        'ft_year2015',
        'ft_year2016',
        'ft_year2017',
        'ft_year2018',
        'ft_year2019',
        'ft_day',
        'ft_hour',
        'age_less_than_14',
        'age_less_than_18',
        'age_18_21',
        'age_21_25',
        'age_25_30',
        'age_30_35',
        'age_35_40',
        'age_40_50',
        'age_more_than_50',
    ]][start:end]
    b_ = len(charisma_ones.columns)

    wallet = data[['product_0_one_hot_0', 'product_0_one_hot_1',
    'product_0_one_hot_2','product_1_one_hot_0','product_1_one_hot_1',
    'product_1_one_hot_2','product_1_one_hot_3','product_2_one_hot_0',
    'product_2_one_hot_1','product_2_one_hot_2','product_2_one_hot_3',
    'product_3_one_hot_0','product_3_one_hot_1','product_3_one_hot_2',
    'product_4_one_hot_0','product_4_one_hot_1','product_4_one_hot_2',
    'product_4_one_hot_3','product_5_one_hot_0','product_5_one_hot_1',
    'product_5_one_hot_2','product_5_one_hot_3','product_6_one_hot_0',
    'product_6_one_hot_1','product_6_one_hot_2','product_6_one_hot_3',
    'marital_status_cd_one_hot_0','marital_status_cd_one_hot_1',
    'marital_status_cd_one_hot_2','marital_status_cd_one_hot_3',
    'marital_status_cd_one_hot_4','marital_status_cd_one_hot_5',
    'marital_status_cd_one_hot_6',
    'job_position_cd_one_hot_1','job_position_cd_one_hot_2','job_position_cd_one_hot_3',
    'job_position_cd_one_hot_4','job_position_cd_one_hot_5','job_position_cd_one_hot_7',
    'job_position_cd_one_hot_8','job_position_cd_one_hot_9','job_position_cd_one_hot_11',
    'job_position_cd_one_hot_12','job_position_cd_one_hot_13','job_position_cd_one_hot_14',
    'job_position_cd_one_hot_15','job_position_cd_one_hot_16','job_position_cd_one_hot_17',
    'job_position_cd_one_hot_18','job_position_cd_one_hot_19','job_position_cd_one_hot_20',
    'job_position_cd_one_hot_21','job_position_cd_one_hot_22' 
    ]][start:end]
    c_ = len(wallet.columns)

    transaq_f = data[transactions.columns[1:]][start:end]
    d_ = len(transaq_f.columns)
    event_ones = data[['e_month1','e_month2','e_month3','e_month4',
    'e_month5','e_month6','e_month7','e_month8','e_month9','e_month10',
    'e_month11','e_month12','e_year2012','e_year2013','e_year2014',
    'e_year2015','e_year2016','e_year2017','e_year2018','e_year2019',
    'e_day','e_hour'
    ]][start:end]
    e_ = len(event_ones.columns)
    event_f = data['story_id_freq'][start:end]
    
    if ev:
        y = LabelEncoder().fit_transform(data['event'][start:end].fillna(0))
        y = to_categorical(y, 4)
        return charisma, charisma_ones, wallet, transaq_f, event_ones, event_f, y
    
    return charisma, charisma_ones, wallet, transaq_f, event_ones, event_f
charisma, charisma_ones, wallet, transaq_f, event_ones, event_f, y = inition(0, 1, data_train)

# Word2Vec (autistic version)

In [7]:
corpus_raw = ' '.join(data_train['name_data'].values).lower()
words_ = []
for word in corpus_raw.split():
    if word != '.': # because we don't want to treat . as a word
        words_.append(word)
words_ = set(words_) # so that all duplicate words are removed
word2int = {}
int2word = {}
vocab_size = len(words_) # gives the total number of unique words
for i,word in enumerate(words_):
    word2int[word] = i
    int2word[i] = word

In [8]:
# function to convert numbers to one hot vectors
def to_one_hot(data_point_index, vocab_size):
    temp = np.zeros(vocab_size, dtype='float32')
    temp[data_point_index] = 1
    return temp

In [9]:
def prepr_sentence(sent):
    sp = [i.lower() for i in sent.split()]
    save = np.array([0 for i in range(vocab_size)], dtype='float32')
    #data_wasted = 0
    for i in range(len(sp)):
        try:
            save += to_one_hot(word2int[sp[i]], vocab_size)
        except KeyError:
            #data_wasted += 1
            save += np.array([0 for i in range(vocab_size)], dtype='float32')
            
    #print(f'wasted data - {data_wasted/save.shape[0]}')
    return save
        
words_data = np.array([prepr_sentence(i) for i in data_train['name_data'].values])
words_data.shape

(935106, 1087)

In [10]:
def basic_model(input_layer, mod=1):
    dense_1 = keras.layers.Dense(16*mod, activation='linear')(input_layer)
    dropout_layer = keras.layers.Dropout(0.5)(dense_1)
    dense_2 = keras.layers.Dense(32*mod, activation='linear')(tf.keras.activations.relu(dropout_layer, alpha=0.1))
    dropout_layer = keras.layers.Dropout(0.5)(dense_2)
    dense_3 = keras.layers.Dense(32*mod, activation='linear')(tf.keras.activations.relu(dropout_layer , alpha=0.1))
    
    return dense_3


def basic_merge(sp, mod=1, add=0):
    merge = keras.layers.Concatenate()(sp)
    dense = keras.layers.Dense(32*mod, activation='linear')(tf.keras.activations.relu(merge, alpha=0.1))
    for i in range(add):
        dense = keras.layers.Dense(16*mod, activation='linear')(tf.keras.activations.relu(dense, alpha=0.1))
    
    return dense

In [ ]:
def complex_model():
    input_charisma = keras.layers.Input(
        (a_), dtype='float32')
    input_charisma_ones = keras.layers.Input(
        (b_), dtype='float32')
    input_wallet = keras.layers.Input(
        (c_), dtype='float32')
    input_transaq_f = keras.layers.Input(
        (d_), dtype='float32')
    input_event_ones = keras.layers.Input(
        (e_), dtype='float32')
    input_event_f = keras.layers.Input(
        (1), dtype='float32')
    
    inp = [input_charisma, input_charisma_ones, input_wallet, input_transaq_f, input_event_ones, input_event_f]
        
    c_dense = basic_model(input_charisma, mod=2)
    co_dense = basic_model(input_charisma_ones, mod=2)
    w_dense = basic_model(input_wallet, mod=3)
    eo_dense = basic_model(input_event_ones)
    ef_dense = basic_model(input_event_f)
    t_dense = basic_model(input_transaq_f, mod=4)
    
    count = len(np.unique(' '.join(data_train['name_data'].unique()).split()))

    input_words = keras.layers.Input(shape=(vocab_size), dtype='float32')
    n_dense_1 = keras.layers.Dense(256, activation='linear')(input_words)
    n_dropout_layer_1 = keras.layers.Dropout(0.5)(n_dense_1)
    n_dense_3 = keras.layers.Dense(128, activation='linear')(tf.keras.activations.relu(n_dropout_layer_1, alpha=0.1))
    n_dropout_layer_2 = keras.layers.Dropout(0.5)(n_dense_3)
    word_dense = keras.layers.Dense(128, activation='linear')(tf.keras.activations.relu(n_dropout_layer_2, alpha=0.1))  
    
    e_merge = basic_merge([eo_dense, ef_dense])
    c_merge = basic_merge([c_dense, co_dense], mod=2)
    ct_merge = basic_merge([c_merge, e_merge], mod=2)
    ctw_merge = basic_merge([word_dense, ct_merge], mod=4, add=1)  
    
    m_merge = basic_merge([t_dense, w_dense], mod=4, add=1)
    mw_merge = basic_merge([word_dense, m_merge], mod=4, add=1)
    
    final_merge = basic_merge([mw_merge, ctw_merge], mod=2, add=1)
    predict_dense = keras.layers.Dense(4, activation='linear')(tf.keras.activations.relu(final_merge, alpha=0.1))
    predict = tf.keras.activations.softmax(predict_dense)
       
    model = keras.Model(
        inputs=inp+[input_words],
        outputs=predict)
    model.compile(optimizer=keras.optimizers.Adam(),
                  loss=tf.losses.categorical_crossentropy,
                  metrics=['accuracy'])
    return model

model = complex_model()

In [16]:
from IPython.display import clear_output
megabatch = 1000
saves = (data_train.shape[0]+megabatch-1)//megabatch
for j in range(200):
    for i in range(saves):
        charisma, charisma_ones, wallet, transaq_f, event_ones, event_f, y = inition(megabatch*i, megabatch*(i+1), data_train)
        words = words_data[megabatch*i: megabatch*(i+1)]
        a = model.fit(
            [charisma.fillna(0).values, 
             charisma_ones.fillna(0).values, 
             wallet.fillna(0).values, 
             transaq_f.fillna(0).values, 
             event_ones.fillna(0).values, 
             event_f.fillna(0).values, 
             words], 
            y,
            batch_size=1024,
            epochs=20,
            validation_split = 0.01,
            use_multiprocessing=True,
            verbose = 0
        )
        b1 = np.mean(a.history['accuracy'])
        b2 = np.mean(a.history['val_accuracy'])
        print(f'accuracy is {b1}, vaildation_accuracy is {b2}')       

ValueError: Dimension 1 in both shapes must be equal, but are 64 and 32. Shapes are [32,64] and [32,32]. for 'Adam/Adam/update_12/ResourceApplyAdam' (op: 'ResourceApplyAdam') with input shapes: [], [], [], [], [], [], [], [], [], [32,64].

In [106]:
model.save_weights('./checkpoints/my_checkpoint')

In [15]:
model.load_weights('./checkpoints/my_checkpoint')

ValueError: Shapes (128,) and (64,) are incompatible

In [ ]:
words_data = np.array([prepr_sentence(i) for i in data_test['name_data'].values])
words_data.shape

In [ ]:
saves = (data_test.shape[0]+megabatch-1)//megabatch
a = []
for i in range(saves):
    words = words_data[megabatch*i: megabatch*(i+1)]
    charisma, charisma_ones, wallet, transaq_f, event_ones, event_f = inition(megabatch*i, megabatch*(i+1), data_test, False)
    a.append(model.predict([charisma.fillna(0).values, 
             charisma_ones.fillna(0).values, 
             wallet.fillna(0).values, 
             transaq_f.fillna(0).values, 
             event_ones.fillna(0).values, 
             event_f.fillna(0).values, 
             words]))